In [2]:
pip install dash sqlalchemy pymysql pandas plotly

   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.9 MB 653.6 kB/s eta 0:00:13
   ---------------------------------------- 0.1/7.9 MB 871.5 kB/s eta 0:00:10
   ---------------------------------------- 0.1/7.9 MB 871.5 kB/s eta 0:00:10
   - -------------------------------------- 0.2/7.9 MB 1.1 MB/s eta 0:00:08
   - -------------------------------------- 0.3/7.9 MB 923.9 kB/s eta 0:00:09
   -- ------------------------------------- 0.4/7.9 MB 1.2 MB/s eta 0:00:07
   -- ------------------------------------- 0.5/7.9 MB 1.4 MB/s eta 0:00:06
   --- ------------------------------------ 0.6/7.9 MB 1.5 MB/s eta 0:00:05
   --- ------------------------------------ 0.7/7.9 MB 1.7 MB/s eta 0:00:05
   --- ------------------------------------ 0.8/7.9 MB 1.5 MB/s eta 0:00:05
   ---- -----------------

DEPRECATION: Loading egg at c:\users\kiit\anaconda3\lib\site-packages\fault_detection-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\kiit\anaconda3\lib\site-packages\pyyaml-6.0-py3.12-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\kiit\anaconda3\lib\site-packages\setuptools-67.6.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [11]:
import pandas as pd
from sqlalchemy import create_engine

# DB Credentials
db_user = 'root'
db_password = 'Ankit%407979'  # '@' encoded as %40
db_host = 'localhost'
db_name = 'airport_db'

# Create Connection Engine
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

# Query Data
query = "SELECT * FROM airports2"
df = pd.read_sql(query, engine)

# Display first few rows
df.head()


,Origin_airport,Destination_airport,Origin_city,Destination_city,Passengers,Seats,Flights,Distance,Fly_date,Origin_population,Destination_population,Org_airport_lat,Org_airport_long,Dest_airport_lat,Dest_airport_long
0,MHK,AMW,"Manhattan, KS","Ames, IA",21,30,1,254,2008-10-01,122049,86219,39.140999,-96.670799,NA,NA
1,EUG,RDM,"Eugene, OR","Bend, OR",41,396,22,103,1990-11-01,284093,76034,44.124599,-123.211998,44.2541008,-121.1500015
2,EUG,RDM,"Eugene, OR","Bend, OR",88,342,19,103,1990-12-01,284093,76034,44.124599,-123.211998,44.2541008,-121.1500015
3,EUG,RDM,"Eugene, OR","Bend, OR",11,72,4,103,1990-10-01,284093,76034,44.124599,-123.211998,44.2541008,-121.1500015
4,MFR,RDM,"Medford, OR","Bend, OR",0,18,1,156,1990-02-01,147300,76034,42.374199,-122.873001,44.2541008,-121.1500015


In [15]:
## Create Simple Dash Application
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine

# DB connection
db_user = 'root'
db_password = 'Ankit%407979'  # '@' encoded
db_host = 'localhost'
db_name = 'airport_db'
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')
query = "SELECT * FROM airports2"
df = pd.read_sql(query, engine)

# Create Dash App
app = Dash(__name__)

# Plot: Passengers vs Distance
fig = px.scatter(df, x='Distance', y='Passengers', color='Origin_airport',
                 title='Passengers Vs Distance', labels={'Passengers': 'Passengers', 'Distance': 'Distance (Miles)'})

# Plot: Seats per flight
fig_bar = px.bar(df, x='Origin_airport', y='Seats', color='Destination_airport',
                 title='Seats per flight', labels={'Seats': 'No of Seats', 'Origin_airport': 'Origin Airport'})

# Layout
app.layout = html.Div(children=[
    html.H1(children='Flight Dashboard'),
    dcc.Graph(
        id='passenger-distance-graph',
        figure=fig
    ),
    dcc.Graph(
        id='seats-bar-chart',
        figure=fig_bar
    ),
    html.Div(children='Dashboard for visualization of flight data')
])

# Run the app
if __name__ == '__main__':
    app.run(debug=True)  # ✅ Use app.run instead of app.run_server


In [20]:
from dash import dcc, html, Input, Output
import dash
import plotly.express as px
import pandas as pd

# Assume df is already loaded
# Example: df = pd.read_csv('airports2.csv')

# Convert Fly_date column to datetime
df['Fly_date'] = pd.to_datetime(df['Fly_date'])

# Create the Dash App
app = dash.Dash(__name__)

# Layout with Dropdown and DatePicker
app.layout = html.Div([
    html.H1("Flight Dashboard"),

    # Dropdown for selecting the Origin Airport
    dcc.Dropdown(
        id='origin-airport-dropdown',
        options=[{'label': airport, 'value': airport} for airport in df['Origin_airport'].unique()],
        placeholder="Select an Origin Airport",
        multi=True
    ),

    # Date range picker for Fly_date
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=df['Fly_date'].min(),
        end_date=df['Fly_date'].max(),
        display_format='YYYY-MM-DD'
    ),

    # Graphs
    dcc.Graph(id='filtered-passenger-distance-graph'),
    dcc.Graph(id='filtered-seats-bar-chart')
])

# Callback for interactivity
@app.callback(
    [Output('filtered-passenger-distance-graph', 'figure'),
     Output('filtered-seats-bar-chart', 'figure')],
    [Input('origin-airport-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_graphs(selected_airports, start_date, end_date):
    # Filter data based on selected airports and date range
    filtered_df = df[(df['Fly_date'] >= start_date) & (df['Fly_date'] <= end_date)]

    if selected_airports:
        filtered_df = filtered_df[filtered_df['Origin_airport'].isin(selected_airports)]

    # Update scatter plot (Passengers vs Distance)
    scatter_fig = px.scatter(
        filtered_df, x='Distance', y='Passengers', color='Origin_airport',
        title="Filtered Passengers vs Distance"
    )

    # Update bar chart (Seats per Origin Airport)
    bar_fig = px.bar(
        filtered_df, x='Origin_airport', y='Seats', color='Destination_airport',
        title="Filtered Seats per Flight"
    )

    return scatter_fig, bar_fig

# ✅ Replace .run_server with .run (Dash v2.14+)
if __name__ == '__main__':
    app.run(debug=True)
